In [ ]:
import torch
import time
import random

## Цель
Обучить нейронную сеть решать шифр Цезаря.

## Напишем алгоритм шифра Цезаря для генерации выборки 

Русский алфавит без буквы ё

In [ ]:
alphabet = ['none'] + [chr(char_code) for char_code in range(ord('а'), ord('я')+1)] + [' ']
MAX_INDEX = len(alphabet)

Самый простой шифр Цезаря

In [ ]:
def caesar_cipher(text, shift=2):
    result = ''
    for char in text:
        shift_index = alphabet.index(char)+shift
        if shift_index>MAX_INDEX-1:
            shift_index -= (MAX_INDEX - 1)
        result += alphabet[shift_index]
    return result

Проверяем как работает

In [ ]:
caesar_cipher('привет саша')

'сткдзфбувъв'

Функция для генерации случайный фраз из случайных букв

In [ ]:
MAX_LEN = 20
def generate_random_sequence(min_length=5, max_length=MAX_LEN):
    sequence_length = random.randint(min_length, max_length)
    sequence = random.choices(alphabet[1:], k=sequence_length)
    return ''.join(sequence)

Функция перевода текста в цифры

In [ ]:
def text_to_int(text):
    result = []
    for char in text:
        result.append(alphabet.index(char))
    return result

Функция перевода списка в тензор

In [ ]:
def set_tensor(list_):
    my_tensor = torch.zeros(len(list_), MAX_LEN, dtype=int)
    for i, item in enumerate(list_):
        my_tensor[i,: len(item)] = torch.tensor(item)
    return my_tensor

Генерируем 1 000 000 фраз
- X - зашифрованная фраза
- y - расшифрованная фраза

In [ ]:
x_list = []
y_list = []
i = 0
while i<1000000:
    y_text = generate_random_sequence()
    x_text = caesar_cipher(y_text)
    x_num = text_to_int(x_text)
    y_num = text_to_int(y_text)
    x_list.append(x_num)
    y_list.append(y_num)
    i += 1
X = set_tensor(x_list)
y = set_tensor(y_list)

## Создаем нейронную сеть

In [ ]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embedding = torch.nn.Embedding(MAX_INDEX, 40)
        self.rnn = torch.nn.RNN(40, 128)
        self.out = torch.nn.Linear(128, MAX_INDEX)

    def forward(self, sentences, state=None):
        x = self.embedding(sentences)
        x, s = self.rnn(x)
        return self.out(x)

## Обучаем ее

In [ ]:
model = Network()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

Так как у нас очень большой датасет, нам будет достаточно 4 эпох

In [ ]:
for ep in range(4):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        X_batch = X[i * 100:(i + 1) * 100]
        Y_batch = y[i * 100:(i + 1) * 100].flatten()

        optimizer.zero_grad()
        answers = model.forward(X_batch)
        answers = answers.view(-1, MAX_INDEX)
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

Epoch 0. Time: 71.426, Train loss: 0.035
Epoch 1. Time: 71.607, Train loss: 0.001
Epoch 2. Time: 71.655, Train loss: 0.001
Epoch 3. Time: 71.444, Train loss: 0.000


Как видно loss 0, значит наша сеть поняла, что мы от нее хотим

## Проверяем качество

Для проверки возьмем другой нормальный текст

In [ ]:
with open('Война и мир.txt') as file:
    text = file.read()
lowercase_text = text.lower()

Возьмем только первые 20 000 символов из нашего словаря, а то книга очень большая

In [ ]:
clear_text = ''
num_space = 0
for i, letter in enumerate(lowercase_text):
    if letter in alphabet[1:-1]:
        num_space = 0
        clear_text += letter
    else:
        if num_space==0:
            clear_text += ' '
        num_space += 1
    if i==20000:
        break

Разделим наш большой текст на подстроки не больше 20 символов до последнего пробела

In [ ]:
def split_text(text, max_length):
    if len(text) <= max_length:
        return [text]

    substrings = []
    start_index = 0

    while start_index < len(text):
        end_index = start_index + max_length

        if end_index >= len(text):
            substrings.append(text[start_index:])
            break

        last_space_index = text.rfind(' ', start_index, end_index)
        
        if last_space_index != -1 and last_space_index > start_index:
            substrings.append(text[start_index:last_space_index].strip())
            start_index = last_space_index + 1
        else:
            substrings.append(text[start_index:end_index].strip())
            start_index = end_index

    return substrings

In [ ]:
substrings = split_text(clear_text, MAX_LEN)

Посчитаем долю совпадений и выведем пару примеров

In [ ]:
matches = 0
for i, input_ in enumerate(substrings):
    coded_text = caesar_cipher(input_)
    answers = model.forward(torch.tensor(text_to_int(coded_text)))
    probas, indices = answers.topk(1)
    output = ''
    for ind in indices.flatten():
        output += alphabet[ind.item()]
    if input_==output:
        matches += 1
    if 20<i<25:
        print(f'ОРИГИНАЛ({input_}) ЗАШИФРОВАННЫЙ ВХОД({coded_text}) РАСШИФРОВАННЫЙ ВЫХОД({output})')
print(f'Доля совпадений: {matches/len(substrings):.0%}')

ОРИГИНАЛ(право я верю что он) ЗАШИФРОВАННЫЙ ВХОД(ствдрбабдзт бщфрбрп) РАСШИФРОВАННЫЙ ВЫХОД(право я верю что он)
ОРИГИНАЛ(антихрист я вас) ЗАШИФРОВАННЫЙ ВХОД(впфкчткуфбабдву) РАСШИФРОВАННЫЙ ВЫХОД(антихрист я вас)
ОРИГИНАЛ(больше не знаю вы) ЗАШИФРОВАННЫЙ ВХОД(грнюъзбпзбйпв бдэ) РАСШИФРОВАННЫЙ ВЫХОД(больше не знаю вы)
ОРИГИНАЛ(уж не друг мой вы) ЗАШИФРОВАННЫЙ ВХОД(хибпзбжтхеборлбдэ) РАСШИФРОВАННЫЙ ВЫХОД(уж не друг мой вы)
Доля совпадений: 100%


Как видно наша сеть отлично расшифровывает текст